# Task

I made this notebook to investigate issues with high model perplexity while reducting loss

I use this model: https://huggingface.co/TheBloke/vicuna-13B-1.1-GPTQ-4bit-128g/tree/main

## Implementation

In [1]:
import torch
import numpy as np

In [2]:
from alpaca_lora_4bit.monkeypatch.peft_tuners_lora_monkey_patch import replace_peft_model_with_int4_lora_model
replace_peft_model_with_int4_lora_model()

c:\Users\alex4321\anaconda3\envs\llama\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Triton not found. Please run "pip install triton".


In [3]:
from alpaca_lora_4bit import autograd_4bit
autograd_4bit.switch_backend_to('cuda')

Using CUDA implementation.


In [4]:
from alpaca_lora_4bit.autograd_4bit import load_llama_model_4bit_low_ram
from peft import LoraConfig, get_peft_model, get_peft_model_state_dict, PeftModel, set_peft_model_state_dict

In [5]:
model, tokenizer = load_llama_model_4bit_low_ram("../vicuna-13b-GPTQ-4bit-128g",
                                                 "../vicuna-13b-GPTQ-4bit-128g/vicuna-13b-4bit-128g.safetensors",
                                                 groupsize=128,
                                                 is_v1_model=False)

Loading Model ...


The safetensors archive passed at ../vicuna-13b-GPTQ-4bit-128g/vicuna-13b-4bit-128g.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.


Loaded the model in 6.70 seconds.


In [6]:
TEXTS = ["""
The vicuña (Lama vicugna) or vicuna[3] (both /vɪˈkuːnjə/, very rarely spelled vicugna, its former genus name) is one of the two wild South American camelids, which live in the high alpine areas of the Andes, the other being the guanaco, which lives at lower elevations.
Vicuñas are relatives of the llama, and are now believed to be the wild ancestor of domesticated alpacas, which are raised for their coats.
Vicuñas produce small amounts of extremely fine wool, which is very expensive because the animal can only be shorn every three years and has to be caught from the wild.
When knitted together, the product of the vicuña's wool is very soft and warm.
The Inca valued vicuñas highly for their wool, and it was against the law for anyone but royalty to wear vicuña garments;
today, the vicuña is the national animal of Peru and appears on the Peruvian coat of arms.

Both under the rule of the Inca and today, vicuñas have been protected by law, but they were heavily hunted in the intervening period.
At the time they were declared endangered in 1974, only about 6,000 animals were left.
Today, the vicuña population has recovered to about 350,000, and although conservation organizations have reduced its level of threat classification, they still call for active conservation programs to protect populations from poaching, habitat loss, and other threats.

Previously the vicuña was thought not to have been domesticated, and the llama and the alpaca were both regarded as descendants of the closely related guanaco.
But DNA research published in 2001 has shown the alpaca may well have vicuña parentage.
Today, the vicuña is mainly wild, but the local people still perform special rituals with these creatures, including a fertility rite.
""".strip(),
"""
The llama (/ˈlɑːmə/; Spanish pronunciation: [ˈʎama]) (Lama glama) is a domesticated South American camelid, widely used as a meat and pack animal by Andean cultures since the Pre-Columbian era.

Llamas are social animals and live with others as a herd.
Their wool is soft and contains only a small amount of lanolin.
Llamas can learn simple tasks after a few repetitions.
When using a pack, they can carry about 25 to 30% of their body weight for 8 to 13 km (5–8 miles).
The name llama (in the past also spelled "lama" or "glama") was adopted by European settlers from native Peruvians.

The ancestors of llamas are thought to have originated from the Great Plains of North America about 40 million years ago, and subsequently migrated to South America about three million years ago during the Great American Interchange.
By the end of the last ice age (10,000–12,000 years ago), camelids were extinct in North America.
As of 2007, there were over seven million llamas and alpacas in South America and over 158,000 llamas and 100,000 alpacas, descended from progenitors imported late in the 20th century, in the United States and Canada.[5]

In Aymara mythology, llamas are important beings.
The Heavenly Llama is said to drink water from the ocean and urinates as it rains.
According to Aymara eschatology, llamas will return to the water springs and lagoons where they come from at the end of time.
""".strip(),
"""
The alpaca (Lama pacos) is a species of South American camelid mammal.
It is similar to, and often confused with, the llama. However, alpacas are often noticeably smaller than llamas.
The two animals are closely related and can successfully crossbreed.
Both species are believed to have been domesticated from their wild relatives, the vicuña and guanaco.
There are two breeds of alpaca: the Suri alpaca and the Huacaya alpaca.

Alpacas are kept in herds that graze on the level heights of the Andes of Southern Peru, Western Bolivia, Ecuador, and Northern Chile at an altitude of 3,500 to 5,000 metres (11,000 to 16,000 feet) above sea level.
Alpacas are considerably smaller than llamas, and unlike llamas, they were not bred to be working animals, but were bred specifically for their fiber.
Alpaca fiber is used for making knitted and woven items, similar to sheep's wool.
These items include blankets, sweaters, hats, gloves, scarves, a wide variety of textiles, and ponchos, in South America, as well as sweaters, socks, coats, and bedding in other parts of the world.
The fiber comes in more than 52 natural colors as classified in Peru, 12 as classified in Australia, and 16 as classified in the United States.

Alpacas communicate through body language.
The most common is spitting to show dominance when they are in distress, fearful, or feel agitated.
Male alpacas are more aggressive than females, and tend to establish dominance within their herd group.
In some cases, alpha males will immobilize the head and neck of a weaker or challenging male in order to show their strength and dominance.

In the textile industry, "alpaca" primarily refers to the hair of Peruvian alpacas, but more broadly it refers to a style of fabric originally made from alpaca hair, such as mohair, Icelandic sheep wool, or even high-quality wool from other breeds of sheep.
In trade, distinctions are made between alpacas and the several styles of mohair and luster.

An adult alpaca generally is between 81 and 99 centimetres (32 and 39 inches) in height at the shoulders (withers).
They usually weigh between 48 and 90 kilograms (106 and 198 pounds).
Raised in the same conditions, the difference in weight can be small with males weighting around 22.3 kilograms (49 lb 3 oz) and females 21.3 kilograms (46 lb 15 oz).
""".strip()
]

In [7]:
from torch.utils.data import Dataset


class ExperimentDataset:
    def __init__(self, encoded_input_ids) -> None:
        self.input_ids = [
            item
            for item in encoded_input_ids
        ]
        self.labels = [
            item
            for item in encoded_input_ids
        ]
    
    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, index):
        return {
            "input_ids": torch.LongTensor(self.input_ids[index]),
            "labels": torch.LongTensor(self.labels[index]),
        }

dataset = ExperimentDataset(encoded_input_ids=[
    tokenizer(text)['input_ids']
    for text in TEXTS
])
len(dataset)

3

In [8]:
def model_perplexity(model, input_ids, labels):
    input_ids = input_ids[:-1]
    labels = labels[1:]
    input_ids_torch = torch.LongTensor([input_ids.tolist()]).to(model.device)
    model.eval()
    with torch.no_grad():
        logits_sentence = model(input_ids_torch).logits[0]
        log_softmax_sentence = torch.nn.functional.log_softmax(logits_sentence, dim=-1)
    log_softmax_sentence_np = log_softmax_sentence.detach().cpu().numpy()
    log_softmax_correct_tokens = np.array([
        log_softmax_sentence_np[i, token]
        for i, token in enumerate(labels.tolist())
    ])
    return np.exp(-log_softmax_correct_tokens.mean())

In [9]:
perplexities_original = [
    model_perplexity(model,
                     **dataset[i])
    for i in range(len(dataset))
]
perplexities_original

[6.494434, 5.0919514, 4.490959]

In [10]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
lora_model = get_peft_model(model, lora_config)

In [11]:
for module in lora_model.modules():
    if 'Autograd4bitQuantLinear' in str(type(module)) or 'Linear4bitLt' in str(type(module)):
        if hasattr(module, "is_v1_model") and module.is_v1_model:
            module.zeros = module.zeros.half()
        module.scales = module.scales.half()

In [12]:
tokenizer.pad_token_id = 0

In [13]:
from alpaca_lora_4bit.gradient_checkpointing import apply_gradient_checkpointing
apply_gradient_checkpointing(lora_model, checkpoint_ratio=1.0);

Forward Patch Applied For Block 0
Forward Patch Applied For Block 1
Forward Patch Applied For Block 2
Forward Patch Applied For Block 3
Forward Patch Applied For Block 4
Forward Patch Applied For Block 5
Forward Patch Applied For Block 6
Forward Patch Applied For Block 7
Forward Patch Applied For Block 8
Forward Patch Applied For Block 9
Forward Patch Applied For Block 10
Forward Patch Applied For Block 11
Forward Patch Applied For Block 12
Forward Patch Applied For Block 13
Forward Patch Applied For Block 14
Forward Patch Applied For Block 15
Forward Patch Applied For Block 16
Forward Patch Applied For Block 17
Forward Patch Applied For Block 18
Forward Patch Applied For Block 19
Forward Patch Applied For Block 20
Forward Patch Applied For Block 21
Forward Patch Applied For Block 22
Forward Patch Applied For Block 23
Forward Patch Applied For Block 24
Forward Patch Applied For Block 25
Forward Patch Applied For Block 26
Forward Patch Applied For Block 27
Forward Patch Applied For Bloc

In [14]:
perplexities_nontuned_lora = [
    model_perplexity(lora_model,
                     **dataset[i])
    for i in range(len(dataset))
]
perplexities_nontuned_lora

[6.494434, 5.0919514, 4.490959]

In [15]:
from transformers import TrainingArguments, Trainer

training_arguments = TrainingArguments(
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    warmup_steps=10,
    optim="adamw_torch",
    num_train_epochs=10,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    evaluation_strategy="epoch",
    save_strategy="no",
    output_dir="test-finetune-loss-perplexity",
    report_to="none",
)

trainer = Trainer(
    model=model,
    train_dataset=dataset,
    eval_dataset=dataset,
    args=training_arguments,
)
model.config.use_cache = False

In [16]:
trainer.evaluate()

100%|██████████| 3/3 [00:02<00:00,  1.41it/s]


{'eval_loss': 1.6666971445083618,
 'eval_runtime': 2.219,
 'eval_samples_per_second': 1.352,
 'eval_steps_per_second': 1.352}

In [17]:
trainer.train()

{'loss': 1.6275, 'learning_rate': 2e-05, 'epoch': 0.33}


{'loss': 1.5018, 'learning_rate': 4e-05, 'epoch': 0.67}


{'loss': 1.8706, 'learning_rate': 6e-05, 'epoch': 1.0}


 67%|██████▋   | 2/3 [00:01<00:00,  1.57it/s]
                                              


{'eval_loss': 1.6609386205673218, 'eval_runtime': 9.042, 'eval_samples_per_second': 0.332, 'eval_steps_per_second': 0.332, 'epoch': 1.0}


...

{'eval_loss': 0.721564769744873, 'eval_runtime': 15.6868, 'eval_samples_per_second': 0.191, 'eval_steps_per_second': 0.191, 'epoch': 10.0}


100%|██████████| 3/3 [00:15<00:00,  2.93s/it]


{'train_runtime': 794.9362, 'train_samples_per_second': 0.038, 'train_steps_per_second': 0.038, 'train_loss': 1.188528655966123, 'epoch': 10.0}


100%|██████████| 30/30 [13:14<00:00, 26.50s/it]


TrainOutput(global_step=30, training_loss=1.188528655966123, metrics={'train_runtime': 794.9362, 'train_samples_per_second': 0.038, 'train_steps_per_second': 0.038, 'train_loss': 1.188528655966123, 'epoch': 10.0})

In [18]:
trainer.evaluate()

100%|██████████| 3/3 [00:15<00:00,  5.28s/it]


{'eval_loss': 0.721564769744873,
 'eval_runtime': 15.9495,
 'eval_samples_per_second': 0.188,
 'eval_steps_per_second': 0.188,
 'epoch': 10.0}

In [19]:
perplexities_nontuned_lora = [
    model_perplexity(lora_model,
                     **dataset[i])
    for i in range(len(dataset))
]
perplexities_nontuned_lora

[2.724882, 1.6267716, 1.964895]

## Results

So:
- initial model perplexities on this texts were: 
    `[6.494379, 5.0919175, 4.490609]`
- perplexities after applying LoRA adapters, but before finetuning (basically the same model with some noise?)
    `[6.494379, 5.0919175, 4.490609]`
- loss of LoRA-applied model before finetuning:
    ```
    {'eval_loss': 13.802539825439453,
    'eval_runtime': 2.9367,
    'eval_samples_per_second': 1.022,
    'eval_steps_per_second': 1.022}
    ```
- loss of LoRA model after trying to overfit for this texts:
    ```
    {'eval_loss': 6.17491340637207,
    'eval_runtime': 26.0902,
    'eval_samples_per_second': 0.115,
    'eval_steps_per_second': 0.115,
    'epoch': 10.0}
    ```
- perplexities
    ```
    [26.089653, 23.530462, 32.777637]
    ```

In [20]:
!pip show peft
!pip show transformers
!pip show alpaca-lora-4bit

Name: peft
Version: 0.3.0
Summary: Parameter-Efficient Fine-Tuning (PEFT)
Home-page: https://github.com/huggingface/peft
Author: The HuggingFace team
Author-email: sourab@huggingface.co
License: Apache
Location: C:\Users\alex4321\anaconda3\envs\llama\Lib\site-packages
Requires: accelerate, numpy, packaging, psutil, pyyaml, torch, transformers
Required-by: alpaca-lora-4bit
Name: transformers
Version: 4.28.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: C:\Users\alex4321\anaconda3\envs\llama\Lib\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, tokenizers, tqdm
Required-by: alpaca-lora-4bit, peft
Name: alpaca-lora-4bit
Version: 0.1.2



Author: 
Author-email: 
License: 
Location: C:\Users\alex4321\anaconda3\envs\llama\Lib\site-packages
Requires: accelerate, colorama, datasets, einops, packaging, peft, pyzmq, safetensors, sentencepiece, torch, transformers, wandb
Required-by: 
